# OPTICS

For the additional we choose optics, the main points of this algorithm is being able to tackle diversity in the density. Also a very nice addition is the possibility of avoiding the usage of $Eps$ given how the algorithm is made and it should somehow overcome the main diversity issue.

All these shoud be able to avoid the shortcomings of DBSCAN.

In [13]:
import datetime
from sklearn.cluster import OPTICS,DBSCAN
from os import path
import pandas as pd
from sklearn.preprocessing import StandardScaler
import utils
from scipy.sparse import csr_matrix

RACES_PATH=path.join("..","dataset","engineered_races.csv")
races_df=pd.read_csv(RACES_PATH)

num_columns = [
            'points',
            'length',
            'climb_total',
            'profile',
            'startlist_quality',
            'position',
            'cyclist_age',
            'delta',
            'climbing_efficiency',
            'competitive_age',
            'difficulty',
            'convenience_score',
            'difficulty_score',
            'performance_index',
            'gain_ratio',
 ]
start_date='2000-12-30'
end_date='2010-00-00'
data_df=races_df.loc[ races_df['date'].between(start_date, end_date),num_columns ].drop_duplicates().astype('float64')
data_df

,points,length,climb_total,profile,startlist_quality,position,cyclist_age,delta,climbing_efficiency,competitive_age,difficulty,convenience_score,difficulty_score,performance_index,gain_ratio
1483,275.0,294000.0,2330.469215,3.0,1400.0,0.0,28.0,0.0,0.007927,28.0,3.190478e+05,0.005220,1.703411,0.657143,161.440822
1484,275.0,294000.0,2330.469215,3.0,1400.0,1.0,34.0,0.0,0.007927,34.0,3.891673e+05,0.004279,1.703411,0.483390,161.440822
1485,275.0,294000.0,2330.469215,3.0,1400.0,2.0,31.0,0.0,0.007927,31.0,1.831936e+06,0.000906,1.703411,0.367874,161.440822
1486,275.0,294000.0,2330.469215,3.0,1400.0,3.0,30.0,0.0,0.007927,30.0,8.173888e+05,0.002036,1.703411,0.450581,161.440822
1487,275.0,294000.0,2330.469215,3.0,1400.0,4.0,31.0,0.0,0.007927,31.0,4.037434e+05,0.004125,1.703411,0.364320,161.440822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587605,50.0,172800.0,2330.469215,3.0,535.0,115.0,26.0,526.0,0.013487,26.0,1.742808e+05,0.001735,1.343767,0.051401,37.208842
587606,50.0,172800.0,2330.469215,3.0,535.0,116.0,31.0,739.0,0.013487,31.0,1.482017e+05,0.002041,1.343767,0.216131,37.208842
587607,50.0,172800.0,2330.469215,3.0,535.0,117.0,24.0,739.0,0.013487,24.0,1.084130e+05,0.002792,1.343767,0.201230,37.208842
587608,50.0,172800.0,2330.469215,3.0,535.0,118.0,26.0,739.0,0.013487,26.0,2.588864e+05,0.001167,1.343767,0.211483,37.208842


In [14]:
from sklearn.preprocessing import StandardScaler

std_scaler=StandardScaler()

std_data=std_scaler.fit_transform(data_df)
#useful to estimatethe density of the dataset
maximum_distance=abs(std_data.max()-std_data.min()).sum().item()
average_concentration= std_data.shape[0]/maximum_distance

print(f"""
maximum distance={maximum_distance}
average concentration={average_concentration}
""")

std_data


maximum distance=50.31084247193086
average concentration=3158.4046736775217



array([[ 3.8430503 ,  2.08271952,  0.03367194, ...,  0.93332655,
         1.98120184, -0.01692766],
       [ 3.8430503 ,  2.08271952,  0.03367194, ...,  0.93332655,
         0.94453857, -0.01692766],
       [ 3.8430503 ,  2.08271952,  0.03367194, ...,  0.93332655,
         0.25533285, -0.01692766],
       ...,
       [-0.63027318,  0.17554477,  0.03367194, ...,  0.23915752,
        -0.73891775, -0.08616871],
       [-0.63027318,  0.17554477,  0.03367194, ...,  0.23915752,
        -0.67774565, -0.08616871],
       [-0.63027318,  0.17554477,  0.03367194, ...,  0.23915752,
        -0.17776565, -0.08616871]])

# 2000 to 2010 plot

So we need to setup the algorithm, i this case we want it to be able to run in reasonable amout of times we have to put a limit on the eps values available for this reason we fix eps at 10%. Given how the algorithms works its not necessary to set a value for eps but it is necessary to set one for the min samples, we apply the same reasoning here without havig to use the kneedle algorithm as in DBSCAN.

So first we take advtange of the algorithm and try a coarse grained approach to clustering.

In [44]:
import time
import itertools as it
from sklearn.metrics import silhouette_score,davies_bouldin_score

n_dim=std_data.shape[1]
n_size=std_data.shape[0]

min_sample_values=[
    #testing by also weighting it with the dimensionality
    int((n_size-1)*0.01*(1/n_dim)),
"""     int((n_size-1)*0.02*(1/n_dim)),
    int((n_size-1)*0.03*(1/n_dim)),
    int((n_size-1)*0.04*(1/n_dim)),
    int((n_size-1)*0.05*(1/n_dim)),
    int((n_size-1)*0.1*(1/n_dim)), """
]

xi_values=[
    0.05,
]
"""     0.1,
    0.2 """
results=[]

for min_sample_size,xi in it.product(min_sample_values,xi_values):
    start=time.time()
    optics=OPTICS(
        min_samples=2 if min_sample_size < 2 else min_sample_size,
        max_eps=maximum_distance*0.1,
        xi=xi,
        n_jobs=-1
    )
    end=time.time()
    optics.fit(std_data)
    labels=optics.labels_
    if not len(set(labels)) > 1:
        silhoutte_val= 'all labels equals'
        davies_bouldin_val= 'all labels equals'
    else:
        silhoutte_val=silhouette_score(std_data,labels)
        davies_bouldin_val=davies_bouldin_score(std_data,labels)
    
    reachability=optics.reachability_
    ordering=optics.ordering_
    core_distances= optics.core_distances_
    res={
            'min_samples':min_sample_size,
            'silhoutte_score':silhoutte_val,
            'davies_bouldin_score':davies_bouldin_val,
            'labels':labels,
            'reachability':reachability,
            'core_distances':core_distances,
            'ordering':ordering,
            'xi':xi,
            'time(s)':(end-start),
        }
    print(res)
    results.append(
        res
    )

results_df=pd.DataFrame(results)

{'min_samples': 105, 'silhoutte_score': 0.6469010608398845, 'davies_bouldin_score': 0.6979715927539477, 'labels': array([0, 0, 0, ..., 0, 0, 0]), 'reachability': array([       inf, 1.21523523, 1.50605628, ..., 0.6539885 , 0.66838638,
       0.67683959]), 'core_distances': array([1.55109873, 1.64596962, 2.08910182, ..., 0.7464875 , 0.83470531,
       0.85837852]), 'ordering': array([     0,      5,     11, ..., 153853, 153854, 153855]), 'xi': 0.05, 'time(s)': 1.4781951904296875e-05}


InvalidParameterError: The 'xi' parameter of OPTICS must be a float in the range [0.0, 1.0]. Got '     0.1,\n    0.2 ' instead.

In [54]:

for k in range(len(results)):
    results[k]['core_distances']=list(map(float,results[k]['core_distances']))
    results[k]['labels']=list(map(float,results[k]['labels']))
    results[k]['reachability']=list(map(float,results[k]['reachability']))
    results[k]['core_distances']=list(map(float,results[k]['core_distances']))
    results[k]['ordering']=list(map(float,results[k]['ordering']))
    results[k]['silhoutte_score']=float(results[k]['silhoutte_score'])
    results[k]['davies_bouldin_score']=float(results[k]['davies_bouldin_score'])
    results[k]['xi']=float(results[k]['xi'])

for k,v in results[0].items():
    print(k,type(v))

min_samples <class 'int'>
silhoutte_score <class 'float'>
davies_bouldin_score <class 'float'>
labels <class 'list'>
reachability <class 'list'>
core_distances <class 'list'>
ordering <class 'list'>
xi <class 'float'>
time(s) <class 'float'>


In [55]:
import json
with open('optics_results_2010to2000b.json','w') as f:
    json.dump(results,f )

In [56]:
top10_results=results_df[results_df['silhoutte_score']!='all labels equals'].sort_values(by='silhoutte_score',ascending=False).head(10)

top10_results

,min_samples,silhoutte_score,davies_bouldin_score,labels,reachability,core_distances,time(s)
3,211,0.801544,1.861952,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.413410263336307, 1.740016527164448, 1....","[1.787429773529056, 1.92280716952867, 2.369388...",0.000016
4,211,0.801544,1.861952,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.413410263336307, 1.740016527164448, 1....","[1.787429773529056, 1.92280716952867, 2.369388...",0.000030
5,211,0.801544,1.861952,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.413410263336307, 1.740016527164448, 1....","[1.787429773529056, 1.92280716952867, 2.369388...",0.000017
6,317,0.800078,1.901963,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.733216946721319, 1.837569921647756, 1....","[1.974289934520991, 2.137637186663265, 2.61703...",0.000008
7,317,0.800078,1.901963,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.733216946721319, 1.837569921647756, 1....","[1.974289934520991, 2.137637186663265, 2.61703...",0.000009
8,317,0.800078,1.901963,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.733216946721319, 1.837569921647756, 1....","[1.974289934520991, 2.137637186663265, 2.61703...",0.000008
9,423,0.798521,1.937091,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.961712450846558, 1.948578790305557, 1....","[2.142167689940046, 2.281491079575051, 2.82560...",0.000011
10,423,0.798521,1.937091,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.961712450846558, 1.948578790305557, 1....","[2.142167689940046, 2.281491079575051, 2.82560...",0.000007
11,423,0.798521,1.937091,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 1.961712450846558, 1.948578790305557, 1....","[2.142167689940046, 2.281491079575051, 2.82560...",0.000010
14,529,0.796925,1.966726,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[inf, 2.151179964575056, 2.064933795097817, 2....","[2.280990230908138, 2.394717277164706, 2.98455...",0.000022


In [38]:
import matplotlib.pyplot as plt
import numpy as np

best_params=top10_results.iloc[0]
print(f"""
best params 
{best_params}
""")
best_optics=OPTICS(
    min_samples= int(2 if best_params['min_samples'] < 2 else best_params['min_samples']),
    max_eps=maximum_distance*0.1,
    n_jobs=-1
)
best_optics.fit(std_data)


best params 
min_samples                                                           211
silhoutte_score                                                  0.801544
davies_bouldin_score                                             1.861952
labels                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
reachability            [inf, 1.413410263336307, 1.740016527164448, 1....
core_distances          [1.787429773529056, 1.92280716952867, 2.369388...
time(s)                                                          0.000016
Name: 3, dtype: object



KeyboardInterrupt: 

In [61]:
import json
results=None
with open('optics_results_2010to2000b.json','r') as f:
    results=json.load(f)
results_df=pd.DataFrame(results).sort_values(by='silhoutte_score')

best_optics=results_df.iloc[0]

In [58]:
labels=best_optics['labels']
reachability=best_optics.reachability_
ordering=best_optics.ordering_
core_distances= best_optics.core_distances_


# Visualize results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

# 1. Reachability plot
ax1.set_title("Reachability Plot")
ax1.plot(np.arange(len(reachability)), reachability[ordering], 'r.', alpha=0.7)
ax1.set_ylabel("Reachability Distance")
ax1.set_xlabel("Ordering of Points")

# 2. Cluster visualization
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

ax2.set_title("Clustered Data")
for k, col in zip(unique_labels, colors):
    if k == -1:
        col = [0, 0, 0, 1]  # Black for noise

    class_member_mask = (labels == k)
    xy = std_data[class_member_mask]
    ax2.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=6)

plt.tight_layout()
plt.show()

# Extract clusters
clusters = {k: np.where(labels == k)[0] for k in unique_labels if k != -1}
print("Extracted Clusters:")
for cluster_id, points in clusters.items():
    print(f"Cluster {cluster_id}: {points}")

AttributeError: 'OPTICS' object has no attribute 'labels_'